In [4]:
!pip install torch-geometric
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.13.1+cu116.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-1.13.1+cu116.html


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.1+cu116.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.1+cu116.html


In [66]:
import pandas as pd
import torch
import networkx as nx
import matplotlib.pyplot as plt

print(torch.__version__)
import torch_geometric
print(torch_geometric.__version__)

1.13.1+cu116
2.2.0


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
!ls drive/MyDrive/elliptic_bitcoin_dataset/

elliptic_txs_classes.csv  elliptic_txs_edgelist.csv  elliptic_txs_features.csv


In [37]:
classes =  pd.read_csv("drive/MyDrive/elliptic_bitcoin_dataset/elliptic_txs_classes.csv")
edgelist = pd.read_csv("drive/MyDrive/elliptic_bitcoin_dataset/elliptic_txs_edgelist.csv")
features = pd.read_csv("drive/MyDrive/elliptic_bitcoin_dataset/elliptic_txs_features.csv", header=None)

features


,0,1,2,3,4,5,6,7,8,9,...,157,158,159,160,161,162,163,164,165,166
0,230425980,1,-0.171469,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162097,...,-0.562153,-0.600999,1.461330,1.461369,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792
1,5530458,1,-0.171484,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162112,...,0.947382,0.673103,-0.979074,-0.978556,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792
2,232022460,1,-0.172107,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162749,...,0.670883,0.439728,-0.979074,-0.978556,-0.098889,-0.106715,-0.131155,-0.183671,-0.120613,-0.119792
3,232438397,1,0.163054,1.963790,-0.646376,12.409294,-0.063725,9.782742,12.414558,-0.163645,...,-0.577099,-0.613614,0.241128,0.241406,1.072793,0.085530,-0.131155,0.677799,-0.120613,-0.119792
4,230460314,1,1.011523,-0.081127,-1.201369,1.153668,0.333276,1.312656,-0.061584,-0.163523,...,-0.511871,-0.400422,0.517257,0.579382,0.018279,0.277775,0.326394,1.293750,0.178136,0.179117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203764,173077460,49,-0.145771,-0.163752,0.463609,-0.121970,-0.043875,-0.113002,-0.061584,-0.135803,...,-0.577099,-0.613614,0.241128,0.241406,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792
203765,158577750,49,-0.165920,-0.123607,1.018602,-0.121970,-0.043875,-0.113002,-0.061584,-0.156418,...,0.162722,0.010822,1.461330,1.461369,-0.098889,-0.087490,-0.084674,-0.140597,-1.760926,-1.760984
203766,158375402,49,-0.172014,-0.078182,1.018602,0.028105,-0.043875,0.054722,-0.061584,-0.163626,...,1.261246,1.985050,1.461330,1.461369,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792
203767,158654197,49,-0.172842,-0.176622,1.018602,-0.121970,-0.043875,-0.113002,-0.061584,-0.163501,...,-0.397749,-0.411776,1.461330,1.461369,-0.098889,-0.087490,-0.084674,-0.140597,1.519700,1.521399


[2.30425980e+08 5.53045800e+06 2.32022460e+08 ... 1.58375402e+08
 1.58654197e+08 1.57597225e+08]
{'class': 'unknown', 'attributes': tensor([ 1.0115e+00, -8.1127e-02, -1.2014e+00,  1.1537e+00,  3.3328e-01,
         1.3127e+00, -6.1584e-02, -1.6352e-01,  4.1399e-02,  9.3589e-01,
        -9.8695e-02,  1.2030e+00,  9.8890e-01,  3.4169e-01, -1.3282e-02,
        -5.7392e-02,  5.8329e-02, -3.2930e-02, -4.2709e-02,  6.5455e-02,
         3.2495e-01, -1.3973e-01, -1.4885e-01, -8.0105e-02, -1.5564e-01,
         1.3131e+00,  1.1070e+00, -1.3973e-01, -1.4885e-01, -8.0104e-02,
        -1.5564e-01,  1.3131e+00,  1.1070e+00, -2.4669e-02, -3.1272e-02,
        -2.3045e-02, -2.6215e-02,  1.4278e-03,  1.4826e-03, -2.2722e-01,
        -2.3936e-01, -7.5242e-02, -2.3495e-01, -8.6606e-01, -8.5603e-01,
        -2.2720e-01, -2.4323e-01, -9.7872e-02, -2.3590e-01, -8.7103e-01,
        -8.6108e-01, -4.1403e-01, -4.8728e-01, -2.3122e-01, -4.6727e-01,
         5.5703e-01,  7.4598e-01, -3.9151e-02, -1.7284e-01, -1.63

In [82]:

G = nx.from_pandas_edgelist(edgelist, source='txId1', target='txId2', create_using=nx.DiGraph())
print(nx.is_directed(G))
G = G.to_undirected(reciprocal=False, as_view=False)
print(nx.is_directed(G))
nx.number_connected_components(G)

nx.set_node_attributes(G, classes.set_index("txId").to_dict("index"))
keys = features.values[:,0]
rows = torch.Tensor(features.values[:, 1:])
dick = dict(zip(keys, rows))
print(keys)
nx.set_node_attributes(G, dick, "attributes")
print(G.nodes[230460314])


True
False
[2.30425980e+08 5.53045800e+06 2.32022460e+08 ... 1.58375402e+08
 1.58654197e+08 1.57597225e+08]
{'class': 'unknown', 'attributes': tensor([ 1.0000e+00,  1.0115e+00, -8.1127e-02, -1.2014e+00,  1.1537e+00,
         3.3328e-01,  1.3127e+00, -6.1584e-02, -1.6352e-01,  4.1399e-02,
         9.3589e-01, -9.8695e-02,  1.2030e+00,  9.8890e-01,  3.4169e-01,
        -1.3282e-02, -5.7392e-02,  5.8329e-02, -3.2930e-02, -4.2709e-02,
         6.5455e-02,  3.2495e-01, -1.3973e-01, -1.4885e-01, -8.0105e-02,
        -1.5564e-01,  1.3131e+00,  1.1070e+00, -1.3973e-01, -1.4885e-01,
        -8.0104e-02, -1.5564e-01,  1.3131e+00,  1.1070e+00, -2.4669e-02,
        -3.1272e-02, -2.3045e-02, -2.6215e-02,  1.4278e-03,  1.4826e-03,
        -2.2722e-01, -2.3936e-01, -7.5242e-02, -2.3495e-01, -8.6606e-01,
        -8.5603e-01, -2.2720e-01, -2.4323e-01, -9.7872e-02, -2.3590e-01,
        -8.7103e-01, -8.6108e-01, -4.1403e-01, -4.8728e-01, -2.3122e-01,
        -4.6727e-01,  5.5703e-01,  7.4598e-01, -3.9151

In [87]:
print(G.nodes[230460314])

A = sorted(nx.connected_components(G), key=len)
S = []
for sub in nx.connected_components(G):
  s = G.subgraph(sub)
  S.append(s)
  nx.set_node_attributes(s, nx.get_node_attributes(s, "class").values())
  

color_map = {"unknown": "grey", 1: "yellow", 2: "red"}
print(S[0].nodes)
colors = [color_map[node["class"]] for node in S[0].nodes]
nx.draw(S[0], node_color=color_map)



{'class': 'unknown', 'attributes': tensor([ 1.0000e+00,  1.0115e+00, -8.1127e-02, -1.2014e+00,  1.1537e+00,
         3.3328e-01,  1.3127e+00, -6.1584e-02, -1.6352e-01,  4.1399e-02,
         9.3589e-01, -9.8695e-02,  1.2030e+00,  9.8890e-01,  3.4169e-01,
        -1.3282e-02, -5.7392e-02,  5.8329e-02, -3.2930e-02, -4.2709e-02,
         6.5455e-02,  3.2495e-01, -1.3973e-01, -1.4885e-01, -8.0105e-02,
        -1.5564e-01,  1.3131e+00,  1.1070e+00, -1.3973e-01, -1.4885e-01,
        -8.0104e-02, -1.5564e-01,  1.3131e+00,  1.1070e+00, -2.4669e-02,
        -3.1272e-02, -2.3045e-02, -2.6215e-02,  1.4278e-03,  1.4826e-03,
        -2.2722e-01, -2.3936e-01, -7.5242e-02, -2.3495e-01, -8.6606e-01,
        -8.5603e-01, -2.2720e-01, -2.4323e-01, -9.7872e-02, -2.3590e-01,
        -8.7103e-01, -8.6108e-01, -4.1403e-01, -4.8728e-01, -2.3122e-01,
        -4.6727e-01,  5.5703e-01,  7.4598e-01, -3.9151e-02, -1.7284e-01,
        -1.6309e-01, -1.6090e-01,  9.4039e-02,  2.2242e-01, -3.9146e-02,
        -1.7283e

AttributeError: ignored

# Nov razdelek